<a href="https://colab.research.google.com/github/JHPark-git/Repo/blob/main/Dacon_%ED%95%9C%EA%B5%AD%EC%96%B4_%EB%AC%B8%EC%9E%A5_%EA%B4%80%EA%B3%84_%EB%B6%84%EC%84%9D_%EA%B2%BD%EC%A7%84%EB%8C%80%ED%9A%8C_%EC%97%B0%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [NLP] NLI/한국어 문장 관계 분석 경진대회 연습 
[휘오/Using Roberta Large](https://dacon.io/competitions/official/235875/codeshare/4597?page=1&dtype=recent), [yuntaeyang/Roberta Large 베이스라인](https://dacon.io/competitions/official/235875/codeshare/4591?page=3&dtype=recent)를 참고해 공부함.

NLI는 두 문장 간의 관계를 entailment, neutral, contradiction의 3 label로 분류하는 작업으로 NLU의 기반.

**Data**

[klue_nli_train](https://klue-benchmark.com/tasks/68/data/description) 24998 set을 train data로, [klue_nli_dev](https://klue-benchmark.com/tasks/68/data/description) 3000 set의 hypothesis를 가공해 제공.

이에 klue_nli_dev data를 train data에 추가해 학습.

NSMC(네이버 영화 감정 분석). airbnb review, wikinews, wikipedia, wikitree, policy로 구성.

NSMC, airbnb review/wikinews, wikipedia, wikitree, policy 두 그룹으로 분류 가능.

두 그룹의 평균 문장 길이, 자주 사용되는 단어, 문장 구성, 문맥 등의 차이점이 존재할 것으로 예상.

추후 그룹 feature를 추가해 학습 모델을 구성해 볼 예정.

**Train**

Roberta-Large 사전학습 모델

모델1: 5fold, 5epochs

모델2: 5fold, 10epochs

모델3: 10fold, 5epochs

모델4: 10fold, 10epochs

4개 모델 seed를 다르게 해 학습후, soft voting으로 ensemble.

In [1]:
!pip install datasets transformers

     |████████████████████████████████| 325 kB 5.4 MB/s 
     |████████████████████████████████| 3.8 MB 30.1 MB/s 
     |████████████████████████████████| 212 kB 38.2 MB/s 
     |████████████████████████████████| 1.1 MB 19.6 MB/s 
     |████████████████████████████████| 67 kB 3.6 MB/s 
     |████████████████████████████████| 134 kB 36.9 MB/s 
     |████████████████████████████████| 127 kB 7.5 MB/s 
     |████████████████████████████████| 596 kB 37.4 MB/s 
     |████████████████████████████████| 895 kB 13.3 MB/s 
     |████████████████████████████████| 6.5 MB 33.2 MB/s 
     |████████████████████████████████| 271 kB 38.2 MB/s 
     |████████████████████████████████| 144 kB 38.9 MB/s 
     |████████████████████████████████| 94 kB 1.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling 

In [2]:
import zipfile
import pandas as pd 
import random
import numpy as np 
import os
import re
import transformers
import torch
import torch.nn as nn
import warnings 
warnings.filterwarnings("ignore")
from collections import Counter
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, AdamW, RobertaForSequenceClassification
from transformers.optimization import get_cosine_schedule_with_warmup

In [3]:
path_to_zip_file = '/content/drive/MyDrive/Colab Notebooks/dataset/한국어문장관계분류.zip'
directory_to_extract_to = '/content'

with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
    zip_ref.extractall(directory_to_extract_to)

**!tar -xzvf**

[klue_dataset](https://klue-benchmark.com/tasks/68/data/description)에서 다운로드한 tar.gz file 압축해제

In [4]:
!tar -xzvf '/content/drive/MyDrive/Colab Notebooks/dataset/klue-nli-v1.1.tar.gz'

klue-nli-v1.1/
klue-nli-v1.1/klue-nli-v1.1_dev.json
klue-nli-v1.1/klue-nli-v1.1_dev_sample_10.json
klue-nli-v1.1/klue-nli-v1.1_train.json


pytorch 실험 결과 재현을 위한 시드 설정

**random.seed(seed)**: python random lib

**np.random.seed(seed)**: numpy lib

**os.environ['PYTHONHASHSEED'] = str(seed)**: python hashing algorithm

**torch.manual_seed(seed)**: pytorch

**torch.cuda.manual_seed/seed_all(seed)**: cuda/multi gpu

**torch.backends.cudnn.deterministic/benchmark = True**: cuda lib cudnn

In [5]:
def seed_everything(seed=42):
  random.seed(seed)
  np.random.seed(seed)
  os.environ['PYTHONHASHSEED']=str(seed)
  torch.manual_seed(seed) # pytorch random seed 설정
  torch.cuda.manual_seed(seed) 
  torch.cuda.manual_seed_all(seed)
  torch.backends.cudnn.deterministic = True #cuda random seed 설정/연산 처리 속도 감소 부작용>마지막 검증단계에 사용
  torch.backends.cudnn.benchmark = False

**전처리**

train_dataset concat 위해 premise, hypothesis, label 항만 추출

**map(dict)**: label to index

**pd.drop_duplicates()**: 중복항 제거



In [6]:
def load_data(train='train_data.csv', test='test_data.csv'):
  train = pd.read_csv('/content/open/'+ train)
  test = pd.read_csv('/content/open/'+ test)
  submission = pd.read_csv('/content/open/sample_submission.csv')
  train_new = pd.read_json('/content/klue-nli-v1.1/klue-nli-v1.1_dev.json')
  train_new= train_new[['premise','hypothesis','gold_label']]
  train_new.columns = ['premise','hypothesis','label']
  train_new = train_new.reset_index()
  train = pd.concat([train, train_new],ignore_index=True)
  label_dict = {'entailment':0,'neutral':1,'contradiction':2}
  train['label'] = train['label'].map(label_dict)
  train.drop_duplicates(inplace=True)
  train = train[['premise','hypothesis','label']]
  test = test[['premise','hypothesis']]
  return train, test, submission

In [7]:
train, test, submission = load_data()

모델1

In [15]:
device = torch.device('cuda')
epochs = 5
batch_size = 8
lr = 1e-4
n_split = 5

모델2

In [ ]:
seed_everything(seed=2002)
device = torch.device('cuda')
epochs = 5
batch_size = 8
lr = 1e-4
n_split = 10

모델3

In [ ]:
seed_everything(seed=3)
device = torch.device('cuda')
epochs = 10
batch_size = 8
lr = 1e-4
n_split = 5

모델4

In [ ]:
seed_everything(seed=50)
device = torch.device('cuda')
epochs = 10
batch_size = 8
lr = 1e-4
n_split = 10

**AutoTokenizer.for_pretrained('klue/roberta-large')(sentences, max_length, pad_to_max_length, add_special_tokens, truncation)**

사전훈련 tokenizer로 입력받은 sentences를 토큰화.

**max_length**: 토큰화 개수

**add_special_tokens**: 시작 토큰[CLS], 문장 맺음 토큰[SEP] 등 추가

**truncation**: max_length보다 문장 토큰의 개수가 많을 시, 절삭 기준

BERT 모델과 달리 Roberta 모델은 token_type_ids 인자 불요

In [9]:
class CustomDataset(Dataset):
  def __init__(self, dataset, option):
    self.dataset = dataset
    self.option = option
    self.tokenizer = AutoTokenizer.from_pretrained('klue/roberta-large')
  
  def __len__(self):
    return len(self.dataset)

  def __getitem__(self, idx):
    row = self.dataset.iloc[idx, 0:3].values
    pre = row[0]
    hyp = row[1]
    inputs = self.tokenizer(
        pre,
        hyp,
        return_tensors='pt',
        max_length = 100,
        pad_to_max_length = True,
        add_special_tokens = True,
        truncation = True)
    
    input_ids = inputs['input_ids'][0]
    attention_mask = inputs['attention_mask'][0]

    if self.option =='train':
      label = row[2]
      return input_ids, attention_mask, label

    return input_ids, attention_mask

평가척도로서 accuracy 함수 설정

**torch.max(tensor, dim)**

dim 기준으로 tensor의 최대값 출력, 출력 format: (max_values, max_indices) tuple

train_acc: 예측이 맞은 항의 개수/전체 항

In [10]:
def calc_accuracy(preds,labels):
  max_values, max_indices = torch.max(preds,1)
  train_acc = (max_indices == labels).sum().data.cpu().numpy()/max_indices.size()[0]
  return train_acc

[RobertaForSequenceClassification](https://kyunghyunlim.github.io/ml_ai/2021/10/04/roberta_cls.html)

**RobertaForSequenceClassification.from_pretrained('klue/roberta-large',num_labels)**

model(token_ids, attention_masks)의 output으로 각 클래스의 logits값을 출력

[transformers optimization](https://huggingface.co/docs/transformers/main_classes/optimizer_schedules#transformers.get_linear_schedule_with_warmup)

**Optimizer**: transformers.AdamW(parameters, lr, weight_decay, correct_bias=False)
=BERTAdam

**Scheduler**: transformers.get_cosine_schedule_with_warmup(optimizer, num_warmup_steps, num_training_steps)

0-초기 설정 lr 사이에서 작동하는 cosine함수에 따라 감소하는 lr scheduler

warmup_steps: 작동하는 스텝 간격

batch마다 scheduler.step()

In [18]:
def training(train_dataset, val_dataset, fold):
  best_acc=0
  model = RobertaForSequenceClassification.from_pretrained('klue/roberta-large',num_labels=3).to(device)
  dataset_train = CustomDataset(train_dataset,option='train')
  dataset_val = CustomDataset(val_dataset,option='train')
  train_loader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
  valid_loader = DataLoader(dataset_val, batch_size=batch_size, shuffle=True)
  optimizer = AdamW(model.parameters(), lr = lr, weight_decay=1e-2, correct_bias=False)
  total_steps = len(train_loader) * epochs
  scheduler = get_cosine_schedule_with_warmup(optimizer,
                                              num_warmup_steps = 1,
                                              num_training_steps=total_steps)
  
  for epoch in range(epochs):
    train_acc = 0.0
    valid_acc = 0.0
    model.train()
    for batch_id,(token_ids, attention_masks, labels)in tqdm(enumerate(train_loader)):
      optimizer.zero_grad()
      token_ids = token_ids.to(device)
      attention_masks = attention_masks.to(device)
      labels = labels.to(device)
      preds = model(token_ids, attention_masks)[0]
      loss = F.cross_entropy(preds, labels)
      loss.backward()
      optimizer.step()
      scheduler.step()
      train_acc += calc_accuracy(preds, labels)

    print('epoch {} train acc {}'.format(epoch+1, train_acc/(batch_id+1)))
    
    model.eval
    for batch_id, (token_ids, attention_masks, labels) in tqdm(enumerate(valid_loader)):
      token_ids = token_ids.to(device)
      attention_masks = attention_masks.to(device)
      labels = labels.to(device)
      preds = model(token_ids, attention_masks)[0]
      valid_acc += calc_accuracy(preds, labels)
    print('epoch {} valid acc {}'.format(epoch+1, valid_acc/(batch_id+1)))
    torch.save(model, '/content/drive/MyDrive/Colab Notebooks/model'+str(fold)+'fold'+str(epochs)+'epochs'+'.pt')

cuda 사용시, memory 부족 문제로 인한 캐시 정리 함수

In [17]:
import gc
gc.collect()
torch.cuda.empty_cache()

**StratifiedKFold(n_splits).split(x,y)**

y를 기준으로 x fold.

pd.DataFrame.iloc vs pd.DataFrame.loc

integer location(index) vs location(label)



In [19]:
folds = []
skf = StratifiedKFold(n_splits=n_split, shuffle=True, random_state=42)
for train_idx, val_idx in skf.split(train.iloc[:,0:2],train.iloc[:,2]):
  folds.append((train.iloc[train_idx,:], train.iloc[val_idx,:]))

for fold, (train_dataset, valid_dataset) in enumerate(folds):
  print(f'fold{fold} 학습중...')
  training(train_dataset = train_dataset, val_dataset=valid_dataset, fold=fold)

fold0 학습중...


Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.bias', 'class

epoch 1 train acc 0.3350297619047619


700it [04:20,  2.69it/s]


epoch 1 valid acc 0.3417857142857143


...

epoch 3 train acc 0.3385119047619048


700it [04:20,  2.69it/s]


epoch 3 valid acc 0.34160714285714283


265it [04:58,  1.13s/it]


KeyboardInterrupt: ignored

In [ ]:
def inference(model, dataset_test):
  test_dataset = CustomDataset(test, option='test')
  test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
  model.eval()
  output_prob = []
  with torch.no_grad():
    for batch_id, (token_ids, attention_masks) in tqdm(enumerate(test_loader), total=len(test_loader)):
      token_ids = token_ids.long().to(device)
      attention_masks = attention_masks.long().to(device)
      output = model(token_ids, attention_masks)[0]
      logits = torch.nn.functional.softmax(output, dim=1).detach().cpu().numpy()
      output_prob.extend(logits)
  return output_prob

In [ ]:
label_dict = {'entailment':0, 'neutral':1, 'contradiction':2}

In [ ]:
def inference_exe(folds, epochs):
  res = np.zeros((len(test),3))
  for fold in range(folds):
    print(f'fold{fold}'+ f'epochs{epochs} 모델 추론중...')
    model = torch.load('/content/drive/MyDrive/Colab Notebooks/model'+str(fold)+'fold'+str(epochs)+'epochs'+'.pt')
    pred_prob = inference(model, test)
    res += np.array(pred_prob)/folds
  return res

In [ ]:
ch5_epochs5   = inference_exe(folds=5, epochs=5)
ch5_epochs10  = inference_exe(folds=5, epochs=10)
ch10_epochs5  = inference_exe(folds=10, epochs=5)
ch10_epochs10 = inference_exe(folds=10, epochs=10)

ensembled_prob = (ch5_epochs5 + ch5_epochs10 + ch10_epochs5 + ch10_epochs10)/4
ans = np.argmax(ensembled_prob, axis=1)
out = [list(label_dict.keys())[_] for _ in ans]
submission['label'] = out

#out = pd.Series(np.argmax(ans,axis=1)).apply(lambda x:(v:k for k,v in label_dict.items())[x])

In [ ]:
submission.to_csv('/content/drive/MyDrive/Colab Notebooks/submission/ensemble_1.csv',index=False)